In [1]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split, StratifiedKFold
from randForest_model import RandFModel
from hyperopt import hp
import send_mail as mail

%matplotlib inline
import mpld3
#mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [334]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

print df.shape
df.head(20)

(2652328, 12)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label
0,2732.521729,0.000000,0.000000,0,0,0,0,0,41004,13721,7490,13721
1,2544.442139,0.000000,0.000000,0,0,0,0,0,29667,26694,7490,13721
2,2538.058105,0.000000,0.000000,0,0,0,0,0,130431,7610,7490,13721
3,2537.544189,0.000000,0.000000,0,0,0,0,0,116675,65463,7490,13721
4,2533.375488,0.000000,0.000000,0,0,0,0,0,127651,17181,7490,13721
5,2527.897705,0.000000,0.000000,0,0,0,0,0,69335,51818,7490,13721
6,2522.609863,0.000000,0.000000,0,0,0,0,0,128410,57058,7490,13721
7,0.000000,4321.461914,0.000000,0,0,0,0,0,78881,5487,7490,13721
8,0.000000,4164.022461,0.000000,0,0,0,0,0,41004,13721,7490,13721
9,0.000000,3976.418945,0.000000,0,0,0,0,0,78720,37921,7490,13721


In [335]:
def similarity(label1, label2):
    if label1 == label2:
        return 1
    else:
        return 0
    
    
df['sim'] = df.apply(lambda row: similarity(row['point_label'], row['probe_label']), axis=1)

df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2732.521729,0,0,0,0,0,0,0,41004,13721,7490,13721,1
1,2544.442139,0,0,0,0,0,0,0,29667,26694,7490,13721,0
2,2538.058105,0,0,0,0,0,0,0,130431,7610,7490,13721,0
3,2537.544189,0,0,0,0,0,0,0,116675,65463,7490,13721,0
4,2533.375488,0,0,0,0,0,0,0,127651,17181,7490,13721,0


In [336]:
#df.to_pickle('formated_train_data_7l.pkl')

In [7]:
df.shape

(7578080, 13)

## Random Forest classifieur

In [2]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

# separating into train and test set in a stratified way
df_train, df_val = train_test_split(df, test_size=0.7, random_state=42, stratify=df['sim'])

print df_train.shape, df_val.shape

(795698, 13) (1856630, 13)


In [5]:
clf = RandomForestClassifier(n_estimators=500, max_depth=10, n_jobs=7, verbose=3)
#clf = RandomForestClassifier(n_estimators=600, max_depth=15, n_jobs=12, verbose=3)

clf.fit(df_train.ix[:,'x1':'x8'], df_train['sim'])

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    3.3s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:   17.5s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:   40.9s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:  1.2min finished


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=7,
            oob_score=False, random_state=None, verbose=3,
            warm_start=False)

In [6]:
print clf.score(df_val.ix[:,'x1':'x8'], df_val['sim'])

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    3.8s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    8.8s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:   16.0s finished


0.950667068829


In [21]:
#joblib.dump(clf, 'randomF_n500_maxD10.pkl')

['randomF_n500_maxD10.pkl',
 'randomF_n500_maxD10.pkl_01.npy',
 'randomF_n500_maxD10.pkl_02.npy',
 'randomF_n500_maxD10.pkl_03.npy',
 'randomF_n500_maxD10.pkl_04.npy',
 'randomF_n500_maxD10.pkl_05.npy',
 'randomF_n500_maxD10.pkl_06.npy',
 'randomF_n500_maxD10.pkl_07.npy',
 'randomF_n500_maxD10.pkl_08.npy',
 'randomF_n500_maxD10.pkl_09.npy',
 'randomF_n500_maxD10.pkl_10.npy',
 'randomF_n500_maxD10.pkl_11.npy',
 'randomF_n500_maxD10.pkl_12.npy',
 'randomF_n500_maxD10.pkl_13.npy',
 'randomF_n500_maxD10.pkl_14.npy',
 'randomF_n500_maxD10.pkl_15.npy',
 'randomF_n500_maxD10.pkl_16.npy',
 'randomF_n500_maxD10.pkl_17.npy',
 'randomF_n500_maxD10.pkl_18.npy',
 'randomF_n500_maxD10.pkl_19.npy',
 'randomF_n500_maxD10.pkl_20.npy',
 'randomF_n500_maxD10.pkl_21.npy',
 'randomF_n500_maxD10.pkl_22.npy',
 'randomF_n500_maxD10.pkl_23.npy',
 'randomF_n500_maxD10.pkl_24.npy',
 'randomF_n500_maxD10.pkl_25.npy',
 'randomF_n500_maxD10.pkl_26.npy',
 'randomF_n500_maxD10.pkl_27.npy',
 'randomF_n500_maxD10.pkl_2

In [2]:
def predict_sim(clf, data, batch_size):
    
    n = int(len(data)/batch_size)+1
    print data.shape, n
    #print data.head()
    
    pred = []
    for i in range(n):
        
        #try:
            print i,'/',n
            print int(batch_size*i), int(batch_size*(i+1))
            #print data.iloc[batch_size*i:batch_size*(i+1),:].head()
            sim = clf.predict(data.iloc[batch_size*i:batch_size*(i+1),:]).tolist()
            #print 'sim', sim
            pred.extend(sim)
            
        #except IndexError:
        #    pass
            
    return pred

In [17]:

predSim = predict_sim(clf, df_val.iloc[:,:8], 1e5)

print len(predSim)
print predSim[:100]

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.9s finished


(2273424, 8) 23
0 / 23
0 100000
1

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    1.0s finished


 / 23
100000 200000
2

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.9s finished


...

 / 23
2000000 2100000
21

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    1.0s finished


 / 23
2100000 2200000
22

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.4s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.7s finished


 / 23
2200000 2300000
2273424
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
df_val['predsim'] = predSim

df_val2 = df_val[df_val['sim']==1]


accuracy = np.mean(df_val2['sim'].values == df_val2['predsim'].values)

print 'accuracy:', accuracy

df_val2.head(40)

accuracy: 0.920953733839


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim,predsim
5304660,0.000000,5739.248535,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,146930,23667,2869,23667,1,1
5304680,0.000000,0.000000,5144.107422,0.000000,0.000000,0.000000,0.000000,0.000000,146930,23667,2869,23667,1,1
5304700,0.000000,0.000000,0.000000,5546.708496,0.000000,0.000000,0.000000,0.000000,146930,23667,2869,23667,1,1
5304720,0.000000,0.000000,0.000000,0.000000,12063.086914,0.000000,0.000000,0.000000,146930,23667,2869,23667,1,1
5304740,0.000000,0.000000,0.000000,0.000000,0.000000,9057.142578,0.000000,0.000000,146930,23667,2869,23667,1,1
5304760,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12059.171875,0.000000,146930,23667,2869,23667,1,1
5304780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5790.572266,146930,23667,2869,23667,1,1
5304802,2545.490234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,84507,10467,145305,10467,1,0
5304820,0.000000,4293.826172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,84507,10467,145305,10467,1,0
5304840,0.000000,0.000000,3507.219727,0.000000,0.000000,0.000000,0.000000,0.000000,84507,10467,145305,10467,1,0


In [329]:
def find_label(probe_id, group):
    #print k, 'probe_id:', name
    
    # finding the predicted similar points and their labels
    try:
        sim_points = group[group['predsim']==1]
        sim_points_labels = sim_points['point_label'].values.astype('int')

        uniques, count = np.unique(sim_points_labels, return_counts=True)
        
        index = np.argwhere(count == np.max(count)).ravel()
        #print 'index', uniques, count, index
        
        if len(index) > 1:
            #print 'multiple max'
            raise ValueError
        else:
            label = uniques[np.argmax(count)]
        
    except ValueError:
        cols = ['x'+str(i+1) for i in range(8)]
        labels = []
        for col in cols:
            dat = group[group[col] != 0]
            #print 'dat1', dat
            dat = dat.iloc[:7,:]
            dat = dat['point_label'].tolist()
            labels.append(dat)
            
        labels = np.asarray(labels)
        
        uniques, count = np.unique(labels, return_counts=True)
        #print 'labels', labels
        #print 'uniques', uniques
        #print 'count', count
        
        try:
            label = uniques[np.argmax(count)]
        except ValueError:
            print group.head()
            print 'labels', labels
            print 'uniques', uniques
            print 'count', count
            label = group['point_label'].iloc[0]
        
        #sim_points_labels = group['point_label'].values.astype('int')
        
        #uniques, count = np.unique(sim_points_labels, return_counts=True)
        #label = uniques[np.argmax(count)]
    
    return {int(probe_id):label}

In [320]:
groups = df_val.groupby('probe_id')

# parallelization
num_cores = multiprocessing.cpu_count()
predLabel = Parallel(n_jobs=num_cores - 1)(delayed(find_label)(probe_id, group) for probe_id, group in groups)
   
predLabel = { k.items()[0][0]: k.items()[0][1] for k in predLabel }
print    
print predLabel

probes_id = df_val['probe_id'].values.astype('int')
probes_label = df_val['probe_label'].values.astype('int')

preds = [predLabel[i] for i in probes_id]

#print preds

accuracy = np.mean(preds == probes_label)


print
print(accuracy)


{0: 31081, 65537: 34794, 2: 36027, 32771: 34458, 14: 30492, 32785: 31126, 98322: 34098, 76462: 34710, 120153: 25031, 32792: 32924, 32794: 27597, 98331: 27421, 32796: 25649, 32797: 28212, 98334: 30567, 98335: 29788, 65570: 29331, 98339: 24722, 43697: 2119, 71004: 32360, 98346: 28107, 32811: 37000, 92850: 25067, 32815: 33213, 98353: 33018, 131125: 29533, 54: 37089, 32825: 34352, 71007: 11551, 131132: 35758, 64: 37126, 65601: 37240, 32834: 32615, 67: 32794, 65604: 23832, 73: 26501, 142007: 25450, 32847: 30650, 98384: 33433, 65617: 33497, 32851: 34495, 84: 32548, 32853: 33230, 136394: 24982, 32858: 30279, 131164: 45252, 131165: 42479, 98400: 29298, 98403: 27484, 32868: 27717, 32869: 36315, 98409: 36187, 65646: 34449, 131183: 35971, 32881: 35753, 98418: 37137, 131187: 46325, 76478: 36823, 131191: 31648, 121: 14470, 65659: 34488, 98428: 27331, 127: 26389, 131202: 29424, 65669: 27160, 98438: 32765, 98440: 24582, 32906: 32530, 65675: 30907, 98444: 36270, 137487: 35010, 145: 37450, 118938: 302

In [141]:
df2 = df_val[df_val['probe_id']==94873]

df2.head(40)

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim,predsim
7002400,2525.979004,0.000000,0,0,0,0,0,0,60997,58518,94873,34278,0,0
7002401,2508.269531,0.000000,0,0,0,0,0,0,80302,35294,94873,34278,0,0
7002402,2505.816162,0.000000,0,0,0,0,0,0,138347,1745,94873,34278,0,0
7002403,2504.257080,0.000000,0,0,0,0,0,0,111030,69302,94873,34278,0,0
7002404,2498.875000,0.000000,0,0,0,0,0,0,49268,57980,94873,34278,0,0
7002405,2496.238281,0.000000,0,0,0,0,0,0,132091,25674,94873,34278,0,0
7002406,2493.981689,0.000000,0,0,0,0,0,0,61736,31682,94873,34278,0,0
7002407,2492.784424,0.000000,0,0,0,0,0,0,139591,60234,94873,34278,0,0
7002408,2491.335449,0.000000,0,0,0,0,0,0,59690,25500,94873,34278,0,0
7002409,2489.859619,0.000000,0,0,0,0,0,0,61067,32226,94873,34278,0,0


### Résultats sur les données test

In [9]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

#df = df[df['predsim']==1]
print df.shape

clf = joblib.load('models/randomF_n500_maxD10.pkl')
print 'model loaded'
print clf.score(df.ix[:1e5,'x1':'x8'], df.ix[:1e5,'sim'])

df.head()

(2652328, 13)
model loaded

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    1.2s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    2.3s finished



0.969000309997


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2732.521729,0,0,0,0,0,0,0,41004,13721,7490,13721,1
1,2544.442139,0,0,0,0,0,0,0,29667,26694,7490,13721,0
2,2538.058105,0,0,0,0,0,0,0,130431,7610,7490,13721,0
3,2537.544189,0,0,0,0,0,0,0,116675,65463,7490,13721,0
4,2533.375488,0,0,0,0,0,0,0,127651,17181,7490,13721,0


In [321]:
df = pd.read_pickle('data/formated_train_data_20l.pkl')

clf = joblib.load('models/randomF_n500_maxD10.pkl')
print 'model loaded'
print clf.score(df.ix[:1e5,'x1':'x8'], df.ix[:1e5,'sim'])

df = pd.read_pickle('data/formated_test_data_20l.pkl')

print df.shape
df.head()

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    1.0s finished


model loaded
0.988870111299
(7577760, 11)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id
0,2866.768066,0,0,0,0,0,0,0,1122820,1008859,1115398
1,2630.219727,0,0,0,0,0,0,0,1136651,1014786,1115398
2,2613.363525,0,0,0,0,0,0,0,1105353,1012033,1115398
3,2593.994141,0,0,0,0,0,0,0,1104138,1017073,1115398
4,2577.335205,0,0,0,0,0,0,0,1157894,1001568,1115398


In [322]:
predSim = predict_sim(clf, df.ix[:,'x1':'x8'], 1e5)

print len(predSim)
print predSim[:100]

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    1.0s finished


(7577760, 8) 76
0 / 76
0 100000
1

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.9s finished


 / 76
100000 200000
2

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.9s finished


...

 / 76
7300000 7400000
74

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.6s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    1.0s finished


 / 76
7400000 7500000
75

[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 114 tasks      | elapsed:    0.3s
[Parallel(n_jobs=7)]: Done 274 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 500 out of 500 | elapsed:    0.8s finished


 / 76
7500000 7600000
7577760
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [323]:
df['predsim'] = predSim

#df.to_csv('test_with_pred_RandF_n500_maxD10.csv')

df.head(20)

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,predsim
0,2866.768066,0,0,0,0,0,0,0,1122820,1008859,1115398,1
1,2630.219727,0,0,0,0,0,0,0,1136651,1014786,1115398,1
2,2613.363525,0,0,0,0,0,0,0,1105353,1012033,1115398,0
3,2593.994141,0,0,0,0,0,0,0,1104138,1017073,1115398,0
4,2577.335205,0,0,0,0,0,0,0,1157894,1001568,1115398,0
5,2572.605469,0,0,0,0,0,0,0,1107498,1016886,1115398,0
6,2566.961914,0,0,0,0,0,0,0,1086109,1012212,1115398,0
7,2550.869385,0,0,0,0,0,0,0,1005950,1017387,1115398,0
8,2542.900635,0,0,0,0,0,0,0,1026012,1015671,1115398,0
9,2540.354248,0,0,0,0,0,0,0,1074204,1019020,1115398,0


In [330]:
groups = df.groupby('probe_id')

# parallelization
num_cores = multiprocessing.cpu_count()
predLabel = Parallel(n_jobs=num_cores - 1)(delayed(find_label)(probe_id, group) for probe_id, group in groups)
   
predLabel = { k.items()[0][0]: k.items()[0][1] for k in predLabel }
print    
print predLabel




{1048579: 1021903, 1048580: 1000029, 1048582: 1042273, 1048585: 1016880, 1048588: 1042180, 1048591: 1009089, 1048593: 1026031.0, 1048599: 1050226, 1143505: 1011346, 1048603: 1007537, 1048608: 1033052, 1048609: 1034801, 1048614: 1031304, 1048615: 1002448, 1048621: 1035279, 1048624: 1008352, 1048627: 1055931, 1048629: 1021597, 1157812: 1015070, 1048634: 1031912, 1048635: 1041867, 1048648: 1033517, 1048649: 1044599, 1157815: 1023207, 1048652: 1046431, 1048659: 1020615, 1048660: 1001324, 1134778: 1002301.0, 1048664: 1029067, 1048667: 1047276, 1048668: 1024058, 1157818: 1001100, 1048671: 1021931, 1048675: 1026480.0, 1048677: 1039374, 1048680: 1022825, 1048683: 1011401, 1048685: 1033007, 1157821: 1027029, 1048688: 1033622, 1048689: 1044132, 1048694: 1020604, 1048695: 1071229, 1048696: 1044260, 1048697: 1005559, 1048698: 1022758, 1048704: 1017338, 1048716: 1012548, 1048717: 1018046, 1048727: 1035588, 1048741: 1017586, 1048742: 1015541, 1048743: 1030002, 1048744: 1037771, 1048756: 1020874, 10

In [331]:
#probes_id = df['probe_id'].values.astype('int')
test = io.loadmat('/home/max/projects/challengeMDI343/data/data_test.mat')
probes_id = test['probeId'][:,0]

print probes_id

preds = np.asarray([predLabel[i] for i in probes_id])

print preds
print preds.shape

np.savetxt('soumission3.txt', preds, fmt='%i')

[1115398 1106004 1003276 ..., 1027010 1158524 1144933]
[ 1008859.  1008860.  1008861. ...,  1044363.  1044364.  1044365.]
(47361,)


In [12]:
df.head(40)

,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,predsim
0,2866.768066,0.000000,0,0,0,0,0,0,1122820,1008859,1115398,1
1,2630.219727,0.000000,0,0,0,0,0,0,1136651,1014786,1115398,1
2,2613.363525,0.000000,0,0,0,0,0,0,1105353,1012033,1115398,0
3,2593.994141,0.000000,0,0,0,0,0,0,1104138,1017073,1115398,0
4,2577.335205,0.000000,0,0,0,0,0,0,1157894,1001568,1115398,0
5,2572.605469,0.000000,0,0,0,0,0,0,1107498,1016886,1115398,0
6,2566.961914,0.000000,0,0,0,0,0,0,1086109,1012212,1115398,0
7,2550.869385,0.000000,0,0,0,0,0,0,1005950,1017387,1115398,0
8,2542.900635,0.000000,0,0,0,0,0,0,1026012,1015671,1115398,0
9,2540.354248,0.000000,0,0,0,0,0,0,1074204,1019020,1115398,0


# RandomForest avec hyperopt

In [7]:
df = pd.read_pickle('data/formated_train_data_7l.pkl')

print df.shape
df.head(20)

(2652328, 13)


,x1,x2,x3,x4,x5,x6,x7,x8,point_id,point_label,probe_id,probe_label,sim
0,2732.521729,0.000000,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
1,2544.442139,0.000000,0.000000,0,0,0,0,0,29667,26694,7490,13721,0
2,2538.058105,0.000000,0.000000,0,0,0,0,0,130431,7610,7490,13721,0
3,2537.544189,0.000000,0.000000,0,0,0,0,0,116675,65463,7490,13721,0
4,2533.375488,0.000000,0.000000,0,0,0,0,0,127651,17181,7490,13721,0
5,2527.897705,0.000000,0.000000,0,0,0,0,0,69335,51818,7490,13721,0
6,2522.609863,0.000000,0.000000,0,0,0,0,0,128410,57058,7490,13721,0
7,0.000000,4321.461914,0.000000,0,0,0,0,0,78881,5487,7490,13721,0
8,0.000000,4164.022461,0.000000,0,0,0,0,0,41004,13721,7490,13721,1
9,0.000000,3976.418945,0.000000,0,0,0,0,0,78720,37921,7490,13721,0


In [8]:
# defining the hyper-parameters space to explore
space = {
             'n_estimators' : hp.quniform('n_estimators', 50, 1000, 1),
             'max_depth' : hp.quniform('max_depth', 1, 15, 1),
             'class_weight' : hp.choice('class_weight', ['balanced', None]),
             'min_samples_split' : hp.quniform('subsample', 2, 10, 1),
             'n_jobs' : 10,
             'verbose': 0
             }


# separating into train and test set in a stratified way
df_train, df_test = train_test_split(df, test_size=0.7, random_state=42, stratify=df['sim'])

print df_train.shape, df_test.shape
print len(df[df['sim']==1])/float(len(df))
print len(df_train[df_train['sim']==1])/float(len(df_train)), len(df_test[df_test['sim']==1])/float(len(df_test)) 

# defining the stratified kfold for cross-validation
skf = StratifiedKFold(df_train['sim'], n_folds=3, random_state=42)


features = ['x'+str(i+1) for i in range(8)]
target = 'sim'

(795698, 13) (1856630, 13)
0.200935555482
0.200935530817 0.200935566052


In [ ]:
model = RandFModel()

model.optimize(space, df_train, features, target, skf, output_file='output_randF_hyperopt.txt')

mail.sendMail('maxime.kubryk@gmail.com','script hyperopt randF ended', 'cool !!')